In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 6.4 MB/s 
     |████████████████████████████████| 7.6 MB 67.4 MB/s 
     |████████████████████████████████| 182 kB 79.8 MB/s 


In [4]:
model_name="roberta-base"

In [5]:
from transformers import AutoModelForSequenceClassification
new_model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/Colab Notebooks/roberta-base/model")

In [6]:
from transformers import AutoTokenizer
new_tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [7]:
import nltk
from nltk.stem.porter import PorterStemmer
import re
from nltk.corpus import stopwords
nltk.download("stopwords")
def preprocessing(text):
    ps = PorterStemmer()
    sentence = re.sub('[^a-zA-Z]',' ',text) # noktalama işaretlerini silme.
    sentence = sentence.lower() # kelimelerin hepsini küçük harf 
    sentence = sentence.split() # keliemeleri listeye atma
    sentence = [ps.stem(kelime) for kelime in sentence if not kelime in set(stopwords.words("english"))] #anlamı olmayan(stopwords) kelimeleri listeden atma
    sentence = " ".join(sentence) # listede kalan kelimeleri birleştirip yeni cümleyi oluşturma.
    return sentence

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
sentence = preprocessing("I wanted to go home.")
print(sentence)

want go home


In [9]:
import torch
import numpy as np

def get_prediction(text):
    text = preprocessing(text)
    encoding = new_tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=128)
    # encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}
    outputs = new_model(**encoding)

    logits = outputs.logits

    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(logits.squeeze().cpu())
    probs = probs.detach().numpy()
    label = np.argmax(probs, axis=-1) # değeri 0 veya 1'e yuvarlıyoruz.
    
    if label == 1:
        print(f"sentiment: True, probability: {probs[1]}")
        return 1
    else:
        print(f"sentiment: Fake, probability: {probs[1]}")
        return 0

In [10]:
get_prediction("He goes to airport.")

sentiment: True, probability: 0.9391817450523376


1

In [11]:
submission = pd.read_csv("/content/drive/MyDrive/submission.csv")
# y = submission["Truth"]
# x = submission["Haber"]

In [12]:
accuracy = 0
y_pred = []
for i in range(submission.shape[0]):
    row = submission.iloc[i]
    text = row[0]
    truth_value = row[1]
    print(f"{i}-)")
    result = get_prediction(text)
    y_pred.append(result)
    if(result == int(truth_value)):
        accuracy += 1 
accuracy

0-)
sentiment: True, probability: 0.9394164085388184
1-)
sentiment: Fake, probability: 0.04899635910987854
2-)
sentiment: Fake, probability: 0.052601248025894165
3-)
sentiment: True, probability: 0.939200758934021
4-)
sentiment: Fake, probability: 0.05130833387374878
5-)
sentiment: True, probability: 0.9394165277481079
6-)
sentiment: True, probability: 0.9393579363822937
7-)
sentiment: Fake, probability: 0.04509388282895088
8-)
sentiment: True, probability: 0.9386608004570007
9-)
sentiment: True, probability: 0.9393612742424011
10-)
sentiment: True, probability: 0.9393813610076904
11-)
sentiment: True, probability: 0.9393183588981628
12-)
sentiment: True, probability: 0.9391000866889954
13-)
sentiment: True, probability: 0.9388035535812378
14-)
sentiment: True, probability: 0.9393751621246338
15-)
sentiment: True, probability: 0.938866913318634
16-)
sentiment: Fake, probability: 0.04777415469288826
17-)
sentiment: True, probability: 0.9390647411346436
18-)
sentiment: Fake, probability:

966

In [13]:
from sklearn.metrics import confusion_matrix

In [14]:
y_true=submission["Truth"].tolist()

In [15]:
conf_matrix=confusion_matrix(y_true, y_pred)
conf_matrix

array([[466,  26],
       [  8, 500]])

In [16]:
TP = conf_matrix[1][1]
TN = conf_matrix[0][0]
FP = conf_matrix[0][1]
FN = conf_matrix[1][0]
print('True Positives:', TP)
print('True Negatives:', TN)
print('False Positives:', FP)
print('False Negatives:', FN)

True Positives: 500
True Negatives: 466
False Positives: 26
False Negatives: 8


In [17]:
conf_accuracy = (float (TP+TN) / float(TP + TN + FP + FN))

conf_error_rate = 1- conf_accuracy
    
conf_sensitivity = (TP / float(TP + FN)) #recall

conf_specificity = (TN / float(TN + FP))
    
conf_precision = (TN / float(TN + FP))

conf_f1 = 2 * ((conf_precision * conf_sensitivity) / (conf_precision + conf_sensitivity))
print('-'*50)
print(f'Accuracy: {round(conf_accuracy,2)}') 
print(f'Error_rate: {round(conf_error_rate,2)}') 
print(f'Sensitivity: {round(conf_sensitivity,2)}') 
print(f'Specificity: {round(conf_specificity,2)}') 
print(f'Precision: {round(conf_precision,2)}')
print(f'f_1 Score: {round(conf_f1,2)}')

--------------------------------------------------
Accuracy: 0.97
Error_rate: 0.03
Sensitivity: 0.98
Specificity: 0.95
Precision: 0.95
f_1 Score: 0.97
